In [3]:
%pip -q install lxml pandas spacy tqdm gender-guesser openpyxl && python -m spacy download de_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
     --------------------------------------- 0.0/14.6 MB 320.0 kB/s eta 0:00:46
     --------------------------------------- 0.0/14.6 MB 326.8 kB/s eta 0:00:45
     --------------------------------------- 0.1/14.6 MB 521.8 kB/s eta 0:00:28
      --------------------------------------- 0.3/14.6 MB 1.6 MB/s eta 0:00:09
     - -------------------------------------- 0.7/14.6 MB 2.6 MB/s eta 0:00:06
     -- ------------------------------------- 0.9/14.6 MB 3.0 MB/s eta 0:00:05
     --- ------------------------------------ 1.3/14.6 MB 3.7 MB/s eta 0:00:04
     ---- ----------------------------------- 1.7/14.6 MB 4.4 MB/s eta 0:00:03
     ------ --------------------------------- 2.2/14.6 MB 5.0 MB/s eta 0:00:03
     ------- -------------------------------- 2.8/14.6 MB 5.7 MB

In [46]:
import os
from lxml import etree
import pandas as pd
from tqdm import tqdm
import spacy
import gender_guesser.detector as gender

# Lade das deutsche Spacy-Modell
nlp = spacy.load("de_core_news_sm")

def get_text_from_alto_xml(file_path):
    """
    Extrahiere Text aus einer AltoXML-Datei mit der lxml-Bibliothek.
    """
    tree = etree.parse(file_path)
    root = tree.getroot()
    
    # Namespace für AltoXML v2
    namespaces = {'alto': 'http://www.loc.gov/standards/alto/ns-v2#'}
    sentences = []
    
    # Gehe durch alle TextBlöcke und extrahiere den Text
    for text_block in root.xpath('//alto:TextBlock', namespaces=namespaces):
        block_text = " ".join([string.get('CONTENT') for string in text_block.xpath('.//alto:String', namespaces=namespaces)])
        doc = nlp(block_text)
        # Füge alle Sätze zum Ergebnis hinzu
        sentences.extend([sent.text.strip() for sent in doc.sents])
    
    return sentences

def get_all_xml_files(directory):
    """
    Durchsuche das Verzeichnis rekursiv nach allen XML-Dateien.
    """
    xml_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".xml"):
                xml_files.append(os.path.join(root, file))
    return xml_files

def extract_names_and_genders(sentences, gender_detector):
    """
    Extrahiere Namen und bestimme deren Geschlecht aus Sätzen mit NER und gender-guesser.
    """
    name_genders = []
    
    # Gehe durch alle Sätze
    for sentence in sentences:
        doc = nlp(sentence)
        # Schaue nach erkannten Entitäten
        for ent in doc.ents:
            if ent.label_ == "PER":  # Wenn die erkannte Entität eine Person ist
                name = ent.text.split()[0]  # Nimm den ersten Teil des Namens (Vorname)
                gender_info = gender_detector.get_gender(name)
                if gender_info != 'unknown':
                    name_genders.append((name, gender_info))
    
    return name_genders

def main(directory):
    """
    Hauptfunktion, die alle Schritte orchestriert.
    """
    # Hole alle XML-Dateien im Verzeichnis
    xml_files = get_all_xml_files(directory)
    all_sentences = []

    print("Verarbeite XML-Dateien:")
    # Verarbeite jede XML-Datei
    for file in tqdm(xml_files, desc="Dateien"):
        sentences = get_text_from_alto_xml(file)
        all_sentences.extend(sentences)
    
    gender_detector = gender.Detector()
    
    print("Extrahiere Namen und Geschlechter:")
    # Extrahiere Namen und Geschlechter aus allen Sätzen
    name_genders = extract_names_and_genders(tqdm(all_sentences, desc="Sätze"), gender_detector)
    
    return name_genders

# Beispielaufruf
directory = "2782137-7 - Die Radlerin"
name_genders = main(directory)

# Konvertiere die Namens- und Geschlechtsinformationen in einen Pandas DataFrame
df = pd.DataFrame(name_genders, columns=['Name', 'Gender'])

# Zähle die Namenshäufigkeiten
df['Frequency'] = df.groupby('Name')['Name'].transform('count')

# Entferne Duplikate
df = df.drop_duplicates(subset=['Name', 'Gender'])

# Sortiere den DataFrame nach Frequenz in absteigender Reihenfolge
df = df.sort_values(by='Frequency', ascending=False)

# Speichere den DataFrame als Excel-Datei
excel_path = "name_frequencies_with_gender.xlsx"
df.to_excel(excel_path, index=False)

# Zeige DataFrame
df

Verarbeite XML-Dateien:


Dateien: 100%|██████████| 896/896 [02:18<00:00,  6.48it/s]


Extrahiere Namen und Geschlechter:


Sätze: 100%|██████████| 53602/53602 [05:52<00:00, 152.02it/s]


,Name,Gender,Frequency
17,Carl,male,364
38,Max,male,338
78,Otto,male,242
185,Paul,male,234
22,Gustav,male,230
...,...,...,...
1797,Al,male,2
7,Balbina,female,2
1829,Dory,female,2
6,Don,male,2
